# 📊 Exploration et Préprocessing des Données - Version Fonctionnelle

Ce notebook fournit une analyse complète du dataset d'actualités avec toutes les fonctionnalités NLP.

## 🎯 Objectifs:
- ✅ Charger et explorer le dataset
- 📈 Analyser les distributions et statistiques
- 🔤 Préprocesser le texte
- 📝 Analyser la fréquence des mots
- ☁️ Créer des visualisations avancées
- 💾 Sauvegarder les données traitées

In [ ]:
# 📦 Import des bibliothèques essentielles
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
import string
from collections import Counter
warnings.filterwarnings('ignore')

# 🎨 Configuration des graphiques
plt.style.use('seaborn-v0_8')
sns.set_palette('viridis')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print('✅ Bibliothèques importées avec succès!')
print('🚀 Prêt pour l\'analyse NLP!')

## 📂 1. Chargement et Exploration du Dataset

In [ ]:
# 📊 Charger le dataset
try:
    df = pd.read_csv('../data/sample/sample_news_data.csv')
    print(f"✅ Dataset chargé avec succès!")
    print(f"📏 Forme du dataset: {df.shape}")
    print(f"📋 Colonnes: {df.columns.tolist()}")
except FileNotFoundError:
    print("❌ Fichier non trouvé. Création d'un dataset d'exemple...")
    # Créer un dataset d'exemple si le fichier n'existe pas
    df = pd.DataFrame({
        'id': range(1, 11),
        'title': [f'Article {i}' for i in range(1, 11)],
        'text': [f'Ceci est le contenu de l\'article {i}. Il contient du texte intéressant pour l\'analyse NLP.' for i in range(1, 11)],
        'category': ['technology', 'sports', 'politics'] * 3 + ['health'],
        'sentiment': ['positive', 'negative', 'neutral'] * 3 + ['positive']
    })
    print(f"✅ Dataset d'exemple créé: {df.shape}")

# 👀 Aperçu des données
print("\n📋 Premières lignes:")
df.head()

In [ ]:
# 🔍 Informations détaillées sur le dataset
print("📊 INFORMATIONS SUR LE DATASET")
print("=" * 50)
print(f"📏 Nombre de lignes: {len(df)}")
print(f"📋 Nombre de colonnes: {len(df.columns)}")
print(f"💾 Taille mémoire: {df.memory_usage(deep=True).sum() / 1024:.2f} KB")

print("\n🔍 Types de données:")
print(df.dtypes)

print("\n❓ Valeurs manquantes:")
missing = df.isnull().sum()
if missing.sum() == 0:
    print("✅ Aucune valeur manquante!")
else:
    print(missing[missing > 0])

print("\n📈 Statistiques descriptives:")
df.describe(include='all')

## 📝 2. Analyse du Texte

In [ ]:
# 📏 Calcul des métriques de texte
print("📊 CALCUL DES MÉTRIQUES DE TEXTE")
print("=" * 40)

# Longueur en caractères
df['text_length'] = df['text'].str.len()

# Nombre de mots
df['word_count'] = df['text'].str.split().str.len()

# Nombre de phrases (approximatif)
df['sentence_count'] = df['text'].str.count(r'[.!?]') + 1

# Nombre de mots uniques
df['unique_words'] = df['text'].apply(lambda x: len(set(x.lower().split())))

# Richesse lexicale (ratio mots uniques / total mots)
df['lexical_diversity'] = df['unique_words'] / df['word_count']

# Statistiques
metrics = ['text_length', 'word_count', 'sentence_count', 'unique_words', 'lexical_diversity']
stats = df[metrics].describe()

print(f"📏 Longueur moyenne: {stats.loc['mean', 'text_length']:.1f} caractères")
print(f"📝 Mots moyens: {stats.loc['mean', 'word_count']:.1f} mots")
print(f"📄 Phrases moyennes: {stats.loc['mean', 'sentence_count']:.1f} phrases")
print(f"🔤 Mots uniques moyens: {stats.loc['mean', 'unique_words']:.1f}")
print(f"🎯 Diversité lexicale moyenne: {stats.loc['mean', 'lexical_diversity']:.3f}")

print("\n📊 Statistiques complètes:")
stats

## 📊 3. Distributions et Visualisations

In [ ]:
# 📈 Visualisation des distributions de texte
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('📊 Distributions des Métriques de Texte', fontsize=16, fontweight='bold')

# Distribution de la longueur
axes[0,0].hist(df['text_length'], bins=15, alpha=0.7, color='skyblue', edgecolor='black')
axes[0,0].axvline(df['text_length'].mean(), color='red', linestyle='--', 
                  label=f'Moyenne: {df["text_length"].mean():.1f}')
axes[0,0].set_title('📏 Distribution de la Longueur (caractères)')
axes[0,0].set_xlabel('Nombre de caractères')
axes[0,0].set_ylabel('Fréquence')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# Distribution du nombre de mots
axes[0,1].hist(df['word_count'], bins=15, alpha=0.7, color='lightgreen', edgecolor='black')
axes[0,1].axvline(df['word_count'].mean(), color='red', linestyle='--',
                  label=f'Moyenne: {df["word_count"].mean():.1f}')
axes[0,1].set_title('📝 Distribution du Nombre de Mots')
axes[0,1].set_xlabel('Nombre de mots')
axes[0,1].set_ylabel('Fréquence')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# Distribution des phrases
axes[1,0].hist(df['sentence_count'], bins=10, alpha=0.7, color='orange', edgecolor='black')
axes[1,0].axvline(df['sentence_count'].mean(), color='red', linestyle='--',
                  label=f'Moyenne: {df["sentence_count"].mean():.1f}')
axes[1,0].set_title('📄 Distribution du Nombre de Phrases')
axes[1,0].set_xlabel('Nombre de phrases')
axes[1,0].set_ylabel('Fréquence')
axes[1,0].legend()
axes[1,0].grid(True, alpha=0.3)

# Diversité lexicale
axes[1,1].hist(df['lexical_diversity'], bins=10, alpha=0.7, color='purple', edgecolor='black')
axes[1,1].axvline(df['lexical_diversity'].mean(), color='red', linestyle='--',
                  label=f'Moyenne: {df["lexical_diversity"].mean():.3f}')
axes[1,1].set_title('🎯 Distribution de la Diversité Lexicale')
axes[1,1].set_xlabel('Ratio mots uniques / total')
axes[1,1].set_ylabel('Fréquence')
axes[1,1].legend()
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 🏷️ Analyse des catégories et sentiments
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Distribution des catégories
category_counts = df['category'].value_counts()
colors1 = plt.cm.Set3(np.linspace(0, 1, len(category_counts)))

bars1 = ax1.bar(category_counts.index, category_counts.values, color=colors1, alpha=0.8, edgecolor='black')
ax1.set_title('🏷️ Distribution des Catégories', fontsize=14, fontweight='bold')
ax1.set_xlabel('Catégorie')
ax1.set_ylabel('Nombre d\'articles')
ax1.tick_params(axis='x', rotation=45)
ax1.grid(True, alpha=0.3)

# Ajouter les valeurs sur les barres
for bar in bars1:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 0.05,
             f'{int(height)}', ha='center', va='bottom', fontweight='bold')

# Distribution des sentiments
sentiment_counts = df['sentiment'].value_counts()
colors2 = {'positive': 'lightgreen', 'negative': 'lightcoral', 'neutral': 'lightblue'}
sentiment_colors = [colors2.get(sent, 'gray') for sent in sentiment_counts.index]

bars2 = ax2.bar(sentiment_counts.index, sentiment_counts.values, color=sentiment_colors, alpha=0.8, edgecolor='black')
ax2.set_title('😊 Distribution des Sentiments', fontsize=14, fontweight='bold')
ax2.set_xlabel('Sentiment')
ax2.set_ylabel('Nombre d\'articles')
ax2.grid(True, alpha=0.3)

# Ajouter les valeurs sur les barres
for bar in bars2:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 0.05,
             f'{int(height)}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

# Afficher les statistiques
print("📊 DISTRIBUTION DES CATÉGORIES:")
for cat, count in category_counts.items():
    percentage = (count / len(df)) * 100
    print(f"  🏷️ {cat}: {count} articles ({percentage:.1f}%)")

print("\n😊 DISTRIBUTION DES SENTIMENTS:")
for sent, count in sentiment_counts.items():
    percentage = (count / len(df)) * 100
    emoji = {'positive': '😊', 'negative': '😞', 'neutral': '😐'}.get(sent, '❓')
    print(f"  {emoji} {sent}: {count} articles ({percentage:.1f}%)")

## 🔤 4. Préprocessing du Texte

In [ ]:
# 🛠️ Fonction de préprocessing avancée
def advanced_preprocess(text):
    """Préprocessing avancé du texte"""
    if not isinstance(text, str):
        return ""
    
    # Convertir en minuscules
    text = text.lower()
    
    # Supprimer les URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # Supprimer les mentions et hashtags
    text = re.sub(r'@\w+|#\w+', '', text)
    
    # Supprimer les chiffres
    text = re.sub(r'\d+', '', text)
    
    # Supprimer la ponctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Supprimer les espaces multiples
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Appliquer le préprocessing
print("🔄 Application du préprocessing...")
df['processed_text'] = df['text'].apply(advanced_preprocess)

# Calculer les nouvelles métriques
df['processed_length'] = df['processed_text'].str.len()
df['processed_words'] = df['processed_text'].str.split().str.len()

print("✅ Préprocessing terminé!")
print(f"📏 Réduction moyenne de longueur: {((df['text_length'] - df['processed_length']) / df['text_length'] * 100).mean():.1f}%")

# Exemples avant/après
print("\n📝 EXEMPLES DE TRANSFORMATION:")
print("=" * 50)
for i in range(min(3, len(df))):
    print(f"\n🔸 Exemple {i+1}:")
    print(f"   📄 Original: {df.iloc[i]['text'][:100]}...")
    print(f"   🔧 Traité: {df.iloc[i]['processed_text'][:100]}...")

## 📊 5. Analyse de Fréquence des Mots

In [ ]:
# 🔍 Analyse de fréquence des mots
print("📊 ANALYSE DE FRÉQUENCE DES MOTS")
print("=" * 40)

# Mots vides français et anglais courants
stop_words = {
    'le', 'de', 'et', 'à', 'un', 'il', 'être', 'et', 'en', 'avoir', 'que', 'pour',
    'dans', 'ce', 'son', 'une', 'sur', 'avec', 'ne', 'se', 'pas', 'tout', 'plus',
    'par', 'grand', 'en', 'être', 'et', 'en', 'avoir', 'que', 'pour', 'du', 'des',
    'the', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'of', 'with', 'by',
    'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had',
    'this', 'that', 'these', 'those', 'a', 'an', 'as', 'if', 'it', 'its'
}

# Extraire tous les mots
all_words = []
for text in df['processed_text']:
    if isinstance(text, str):
        words = text.split()
        # Filtrer les mots courts et les mots vides
        words = [word for word in words if len(word) > 2 and word not in stop_words]
        all_words.extend(words)

# Compter les fréquences
word_freq = Counter(all_words)
top_words = word_freq.most_common(20)

print(f"📝 Total de mots uniques: {len(word_freq)}")
print(f"📊 Total d'occurrences: {sum(word_freq.values())}")

print("\n🏆 TOP 20 DES MOTS LES PLUS FRÉQUENTS:")
for i, (word, freq) in enumerate(top_words, 1):
    percentage = (freq / sum(word_freq.values())) * 100
    print(f"  {i:2d}. {word:<15} : {freq:3d} fois ({percentage:.1f}%)")

In [ ]:
# 📊 Visualisation de la fréquence des mots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))

# Graphique en barres horizontales
words, frequencies = zip(*top_words)
colors = plt.cm.viridis(np.linspace(0, 1, len(words)))

bars = ax1.barh(range(len(words)), frequencies, color=colors, alpha=0.8, edgecolor='black')
ax1.set_yticks(range(len(words)))
ax1.set_yticklabels(words)
ax1.set_xlabel('Fréquence')
ax1.set_title('🏆 Top 20 des Mots les Plus Fréquents', fontsize=14, fontweight='bold')
ax1.invert_yaxis()
ax1.grid(True, alpha=0.3, axis='x')

# Ajouter les valeurs sur les barres
for i, bar in enumerate(bars):
    width = bar.get_width()
    ax1.text(width + 0.1, bar.get_y() + bar.get_height()/2,
             f'{int(width)}', ha='left', va='center', fontweight='bold')

# Distribution de Zipf (loi de puissance)
ranks = np.arange(1, len(top_words) + 1)
ax2.loglog(ranks, frequencies, 'bo-', alpha=0.7, linewidth=2, markersize=8)
ax2.set_xlabel('Rang (log)')
ax2.set_ylabel('Fréquence (log)')
ax2.set_title('📈 Distribution de Zipf', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

# Ligne de tendance
z = np.polyfit(np.log(ranks), np.log(frequencies), 1)
p = np.poly1d(z)
ax2.plot(ranks, np.exp(p(np.log(ranks))), "r--", alpha=0.8, linewidth=2, 
         label=f'Pente: {z[0]:.2f}')
ax2.legend()

plt.tight_layout()
plt.show()

print(f"📊 Coefficient de Zipf: {abs(z[0]):.2f} (idéal ≈ 1.0)")
if abs(z[0]) > 0.8:
    print("✅ Distribution proche de la loi de Zipf (texte naturel)")
else:
    print("⚠️ Distribution éloignée de la loi de Zipf")

## ☁️ 6. Nuage de Mots et Visualisations Avancées

In [ ]:
# ☁️ Création de nuages de mots
try:
    from wordcloud import WordCloud
    
    # Combiner tout le texte préprocessé
    combined_text = ' '.join(df['processed_text'].dropna())
    
    # Créer le nuage de mots principal
    wordcloud = WordCloud(
        width=1200, 
        height=600, 
        background_color='white',
        max_words=100,
        colormap='viridis',
        relative_scaling=0.5,
        min_font_size=10
    ).generate(combined_text)
    
    plt.figure(figsize=(15, 8))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title('☁️ Nuage de Mots Global', fontsize=16, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.show()
    
    # Nuages de mots par sentiment
    sentiments = df['sentiment'].unique()
    if len(sentiments) > 1:
        fig, axes = plt.subplots(1, len(sentiments), figsize=(5*len(sentiments), 6))
        if len(sentiments) == 1:
            axes = [axes]
        
        colors = {'positive': 'Greens', 'negative': 'Reds', 'neutral': 'Blues'}
        
        for i, sentiment in enumerate(sentiments):
            sentiment_text = ' '.join(df[df['sentiment'] == sentiment]['processed_text'].dropna())
            
            if sentiment_text.strip():  # Vérifier que le texte n'est pas vide
                wc = WordCloud(
                    width=400, height=300,
                    background_color='white',
                    max_words=50,
                    colormap=colors.get(sentiment, 'viridis')
                ).generate(sentiment_text)
                
                axes[i].imshow(wc, interpolation='bilinear')
                axes[i].axis('off')
                emoji = {'positive': '😊', 'negative': '😞', 'neutral': '😐'}.get(sentiment, '❓')
                axes[i].set_title(f'{emoji} {sentiment.title()}', fontsize=12, fontweight='bold')
        
        plt.suptitle('☁️ Nuages de Mots par Sentiment', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()
    
    print("✅ Nuages de mots créés avec succès!")
    
except ImportError:
    print("⚠️ WordCloud non disponible. Affichage textuel des mots fréquents:")
    print("\n" + "="*60)
    for word, freq in top_words[:15]:
        bar_length = int((freq / max(frequencies)) * 40)
        bar = "█" * bar_length
        print(f"{word:<15} {bar} {freq}")
    print("="*60)

## 💾 7. Sauvegarde et Résumé

In [ ]:
# 💾 Sauvegarde des données préprocessées
import os

# Créer le dossier processed s'il n'existe pas
os.makedirs('../data/processed', exist_ok=True)

# Sauvegarder le dataset enrichi
output_file = '../data/processed/processed_news_data.csv'
df.to_csv(output_file, index=False)

print(f"💾 Données sauvegardées dans: {output_file}")
print(f"📊 Taille du fichier: {os.path.getsize(output_file) / 1024:.2f} KB")

# Sauvegarder les statistiques de mots
word_stats = pd.DataFrame(top_words, columns=['word', 'frequency'])
word_stats['percentage'] = (word_stats['frequency'] / sum(word_freq.values())) * 100
word_stats.to_csv('../data/processed/word_frequencies.csv', index=False)

print("📝 Fréquences des mots sauvegardées dans: ../data/processed/word_frequencies.csv")

In [ ]:
# 📋 Résumé final de l'analyse
print("\n" + "="*60)
print("🎉 RÉSUMÉ DE L'ANALYSE EXPLORATOIRE")
print("="*60)

print(f"\n📊 DATASET:")
print(f"   📏 {len(df)} articles analysés")
print(f"   🏷️ {df['category'].nunique()} catégories: {', '.join(df['category'].unique())}")
print(f"   😊 {df['sentiment'].nunique()} sentiments: {', '.join(df['sentiment'].unique())}")

print(f"\n📝 MÉTRIQUES DE TEXTE:")
print(f"   📏 Longueur moyenne: {df['text_length'].mean():.1f} caractères")
print(f"   🔤 Mots moyens: {df['word_count'].mean():.1f} mots")
print(f"   📄 Phrases moyennes: {df['sentence_count'].mean():.1f} phrases")
print(f"   🎯 Diversité lexicale: {df['lexical_diversity'].mean():.3f}")

print(f"\n🔍 ANALYSE LEXICALE:")
print(f"   📚 {len(word_freq)} mots uniques identifiés")
print(f"   🏆 Mot le plus fréquent: '{top_words[0][0]}' ({top_words[0][1]} fois)")
print(f"   📊 Coefficient de Zipf: {abs(z[0]):.2f}")

print(f"\n✅ FICHIERS GÉNÉRÉS:")
print(f"   💾 Dataset préprocessé: processed_news_data.csv")
print(f"   📊 Fréquences des mots: word_frequencies.csv")
print(f"   📈 Visualisations: Graphiques interactifs générés")

print(f"\n🚀 PROCHAINES ÉTAPES RECOMMANDÉES:")
print(f"   1️⃣ Analyse de sentiment avec TextBlob/VADER")
print(f"   2️⃣ Modélisation de sujets avec LDA/BERTopic")
print(f"   3️⃣ Reconnaissance d'entités nommées (NER)")
print(f"   4️⃣ Résumé automatique de texte")
print(f"   5️⃣ Classification automatique")

print("\n" + "="*60)
print("🎯 ANALYSE TERMINÉE AVEC SUCCÈS!")
print("="*60)